## Packages

In [1]:
%load_ext Cython

# Single-Pivot Network Simplex Method

In [2]:
%%cython

from libc.math cimport sqrt, ceil
#from math import sqrt, ceil

#####################################################
# Update flows in passed Cycle
#####################################################
def update_flows(cycle, flow, x):
    for arc in cycle: x[arc[:2]] += flow if arc[2] else -flow
        
def artifical_nodes_and_arcs(nodes, x, u, c, demand, arcs):
    M = sum(c[arc] for arc in arcs)
    
    #Setup Artifical Node
    nodes += 1
    
    #make the artificial arc the root
    p = [-1] + [nodes]*(nodes-1) + [0]          #predecessors
    s =  [0] + [1]*(nodes-1) + [nodes]          #successors
    e =  [0] + list(range(1,nodes)) + [nodes-1] #last node of tree rooted at e
    t =  [0] + list(range(2,nodes+1)) + [1]     #thread
    rt=  [0, nodes] + list(range(1,nodes))      #reverse thread
    w =  [0]*(nodes+1)                          #node potentials
    d =  [0]*(nodes+1)                          #direction
    d[nodes] = -1
    
    for index, dem in enumerate(demand):
        artifical_arc = (index + 1, nodes) if dem >= 0 else (nodes, index + 1)
        w[index + 1] = M if dem >= 0 else 0
        d[index + 1] = dem >= 0
        u[artifical_arc] = float("inf")
        x[artifical_arc] = abs(dem)
        
    return w,p,s,e,t,d,rt

def find_cycle(d,p,s,u,x, lowerbound, entering_arc):
    def compare(arc):
        if not arc[2] and x[arc[:2]] < min_arc[1]: return (arc,x[arc[:2]])
        elif arc[2] and u[arc[:2]] - x[arc[:2]] < min_arc[1]: return (arc,u[arc[:2]] - x[arc[:2]]) 
        else: return min_arc
    
    i,j = entering_arc
    cycle = [(i,j, lowerbound)]
    min_arc = (cycle[-1], u[entering_arc] - x[entering_arc] if lowerbound else x[entering_arc])
    
    while i != j:
        if s[i] > s[j]:
            
            cycle.append((j,p[j], lowerbound)  if d[j] else (p[j],j, not lowerbound))
            j = p[j]
            
        elif s[i] < s[j]:
            cycle.append((i,p[i], not lowerbound) if d[i] else (p[i],i, lowerbound))
            i = p[i]
            
        else:
            cycle.append((j,p[j], lowerbound) if d[j] else (p[j],j, not lowerbound))
            min_arc = compare(cycle[-1])
            cycle.append((i,p[i], not lowerbound) if d[i] else (p[i],i, lowerbound))
                
            i, j = p[i], p[j]
        
        min_arc = compare(cycle[-1])
        
    return cycle, min_arc[0], min_arc[1]

def update_subtree(s, pr, t, e, w, d, rt, cpiv, entering_arc, leaving_arc, root_node):
    
    #finds nodes from a source to target dynamically
    def path(source, target):
        nodes = []
        while source != target:
            nodes.append(source)
            source = pr[source]
        
        return nodes + [source]
    
    def update_potentials(cpiv, y2, x2):    
        w[x2] += cpiv
        x = t[x2]
        
        while x != x2:
            w[x] += cpiv
            x = t[x]
    #-----------------------------------------------------------------------------
    
    def Step_One(q,p,v,u):
        #
        # Update T-T(q)
        #
        # No p(x) update required
        # Change threading q
        #
        y = rt[q]
        t[y] = t[e[q]]
        rt[t[e[q]]] = y
        #
        #Update ending nodes e()
        #
        p_root = path(p, root_node)
        xp = p_root if pr[t[e[q]]] == 0 else path(p, pr[t[e[q]]])[:-1] 
        
        for x in xp: e[x] = y  #Update End Nodes
        for x in p_root: s[x] -= s[q] #Update sucessors
        #
        # Update T(q)
        #
        pr[q] = 0
        t[e[q]] = q
        rt[q] = e[q]
        d[q] = -1
        
    def Step_Two(q, v, u):
        #update T - T(q) (T1 = T(q), T2 = T - T(q))
        x1, x2 = q, root_node
        y1, y2 = v, u

        #else if the T - T(q) is bigger, update T(q) (T1 = T - T(q), T2 = T(q))
        if s[q] <= s[root_node]: x1,y1,x2,y2 = x2,y2,x1,y1
        
        return x1, x2, y1, y2

    #Parts A, B, and C can be combined into their resepective character assignments
    def Step_Three(x2, y2):
        
        y2x2path = path(y2,x2)[::-1]
        
        #Treat every iteration from y2 to x2 as it's own subtree and reverse
        #the entire tree from there
        for current_x, next_x in zip(y2x2path, y2x2path[1:]):
            #####################################################
            # Part A - Initilization Variables to allow for concurrent modificaiton of arrays
            #####################################################
            size_current_x = s[current_x]
            end_current_x = e[current_x]
            rt_next_x = rt[next_x]
            end_next_x = e[next_x]
            thread_next_x = t[end_next_x]
            d[current_x] = not d[next_x]
            
            #####################################
            # Part B.1
            #####################################
            # Reverses the current node with its predessesor
            # and calculate successors
            pr[current_x] = next_x
            pr[next_x] = 0
            s[current_x] -= s[next_x]
            s[next_x] = size_current_x
            
            #############################################
            # Part B and C - Reverse threading and change ending nodes
            ############################################
            t[rt_next_x] = thread_next_x
            rt[thread_next_x] = rt_next_x
            t[end_next_x] = t[end_current_x] = next_x
            rt[next_x] = rt[current_x] = end_next_x
            if end_current_x == end_next_x: e[current_x] = end_current_x = rt_next_x
            t[end_next_x] = current_x
            rt[next_x] = e[next_x] = end_current_x
        
################################################################################################################
#-----START----------------------------------------------------------------------------------------------------#
################################################################################################################
    p,q = leaving_arc if pr[leaving_arc[1]] == leaving_arc[0] else leaving_arc[::-1]
    u,v = entering_arc if q in path(entering_arc[1], root_node) else entering_arc[::-1]
    
    Step_One(q,p,v,u)
    x1, x2, y1, y2 = Step_Two(q,v,u)
    
    if y2 == entering_arc[0]: cpiv *= -1
    
    update_potentials(cpiv, y2, x2)
    
    if x2 != y2: Step_Three(x2, y2)
        
    #~_~_~_~_~_~_~_Attaching Arc back to Tree~_~_~_~_~_~_~_~_~_~
    
    thread_y1 = t[e[y2]] = t[y1]
    rt[t[y1]] = e[y2]
    t[y1] = y2
    rt[y2] = y1
    pr[y2] = y1
    d[y2] = entering_arc[1] == y1
    
    #
    # Update Ending Nodes & Update Successors
    # 
    if (x_bar:=pr[thread_y1]) != 0:
        hit_xbar = False
        
        for x in path(y1, x1):
            if x != x_bar and not hit_xbar: e[x] = e[y2]
            else: hit_xbar = True
            s[x] += s[y2]
            
    else:
        for x in path(y1, x1):
            e[x] = e[y2]
            s[x] += s[y2]
            
    return x1

def entering_arcs(w,c,x, arcs):
    
    def cbar_enter(arc): return  cbar(arc) if x[arc] == 0 else -cbar(arc)
    
    def cbar(arc): return w[arc[0]] - w[arc[1]] - c[arc]
    
    number_of_NBV = len(arcs)
    block = int(ceil(sqrt(number_of_NBV)))
    total_blocks = (number_of_NBV + block - 1) // block
    
    first_edge, number_of_blocks = 0, 0
    
    while total_blocks > number_of_blocks:
        last_edge = first_edge + block

        if last_edge <= number_of_NBV: canidates = arcs[first_edge: last_edge]
        else:
            last_edge -= number_of_NBV
            canidates =  arcs[:last_edge] + arcs[first_edge:number_of_NBV]

        first_edge = last_edge

        if  cbar_enter(arc:=max(canidates, key=lambda arc: cbar_enter(arc))) > 0:
            number_of_blocks=0
            yield arc
        else:
            number_of_blocks += 1
            
def network_simplex(nodes, arcs, x, u, demand, c):
    
    def cbar_enter(arc): return  cbar(arc) if x[arc] == 0 else -cbar(arc)
    
    def cbar(arc): return w[arc[0]] - w[arc[1]] - c[arc]
    
    w,p,s,e,t,d,rt = artifical_nodes_and_arcs(nodes, x, u, c, demand, arcs)
    root = nodes + 1
    
    #Start Pivoting
    for improving_arc in entering_arcs(w,c,x, arcs):
        #Find a Cycle and Leaving arc w/ flow
        cycle, leaving_arc, flow = find_cycle(d,p,s,u,x, x[improving_arc] == 0 , improving_arc)
        
        if flow != 0: update_flows(cycle, flow, x)
            
        if improving_arc != leaving_arc[:2]: root = update_subtree(s,p,t,e,w,d,rt, cbar(improving_arc), improving_arc, leaving_arc[:2], root)
            
    return sum([x[arc]*c[arc] for arc in arcs])